# Set Up

In [1]:
import time

import tensorflow as tf
from tensorflow.keras import layers, Sequential

from utils import images_path

tf.config.list_physical_devices('GPU')

[]

In [21]:
batch_size = 64
seed = int(time.time())
train_ds = tf.keras.utils.image_dataset_from_directory(
    images_path,
    validation_split=0.4,
    subset= "training",
    seed= seed,
    image_size= (360, 640),
    batch_size= batch_size,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    images_path,
    validation_split=0.2,
    subset= "validation",
    seed= seed,
    image_size= (360, 640),
    batch_size= batch_size,
)

val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 2)
val_ds = val_ds.skip(val_batches // 2)

Found 5773 files belonging to 2 classes.
Using 3464 files for training.
Found 5773 files belonging to 2 classes.
Using 1154 files for validation.


In [22]:
model = Sequential(
    [
        layers.Conv2D(6, 5, padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        layers.Conv2D(16, 5, padding="valid", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        layers.Flatten(),
        layers.Dense(1024, activation="relu"),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(1, activation= "linear")
    ]
)

In [23]:
lr = 0.001
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate= lr),
    loss= tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics= ["accuracy",],
)

In [24]:
epochs = 3
model.fit(
  train_ds,
  validation_data= val_ds,
  epochs= epochs
)

Epoch 1/3


55/55 [==============================] - 1036s 18s/step - loss: 76.1467 - accuracy: 0.9397 - val_loss: 0.0983 - val_accuracy: 0.9913
Epoch 2/3
55/55 [==============================] - 858s 15s/step - loss: 0.0477 - accuracy: 0.9945 - val_loss: 0.1319 - val_accuracy: 0.9965
Epoch 3/3
55/55 [==============================] - 940s 17s/step - loss: 0.0200 - accuracy: 0.9983 - val_loss: 0.0860 - val_accuracy: 0.9983


In [27]:
loss = model.evaluate(test_ds)
loss

9/9 [==============================] - 43s 4s/step - loss: 0.0131 - accuracy: 0.9965


[0.013139743357896805, 0.9965277910232544]

In [ ]:
model_name = f"tf_model_{epochs=}_{batch_size=}_{lr=}_{loss[0]=:.4f}.keras"
model.save(model_name)